In [ ]:
"google/vit-base-patch16-224-in21k", "facebook/deit-base-patch16-224", "microsoft/swin-base-patch4-window7-224", "microsoft/beit-base-patch16-224-pt22k-ft22k"

In [1]:
import os
import random
import numpy as np
import torch

def seed_everything(seed=42):
    """
    Python, Numpy, PyTorch ve cuDNN kütüphanelerini
    aynı 'seed' ile sabitler.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Çoklu GPU varsa
    
    # PyTorch determinism ayarları
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # (Opsiyonel) Reproducibility log
    print(f"[INFO] Global seed set to {seed}")

seed_everything()

[INFO] Global seed set to 42


In [2]:
##############################################
# EĞİTİM KODU (Z-Score + Haversine + AMP + Optuna + CosineAnnealingLR + 2 GPU)
# Ek: Daha Fazla Print & TQDM (Debug Logging)
##############################################

import os
import math
import time
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

# transformers kütüphanesi
from transformers import AutoConfig, AutoModel
from tqdm import tqdm

# Automatic Mixed Precision
from torch.cuda.amp import autocast, GradScaler

# Optuna
import optuna

#######################
# 1) Z-SCORE İSTATİSTİKLERİ
#######################
LAT_MEAN = 41.105763
LAT_STD  = 0.002358
LON_MEAN = 29.025191
LON_STD  = 0.004205

def zscore_lat_lon(lat, lon):
    lat_norm = (lat - LAT_MEAN) / LAT_STD
    lon_norm = (lon - LON_MEAN) / LON_STD
    return lat_norm, lon_norm

def inverse_zscore_lat_lon(lat_norm, lon_norm):
    lat_real = lat_norm * LAT_STD + LAT_MEAN
    lon_real = lon_norm * LON_STD + LON_MEAN
    return lat_real, lon_real

#######################
# 2) HAVERSINE MESAFE HESABI
#######################
def haversine_distance_tensor(lat_pred, lon_pred, lat_true, lon_true):
    R = 6371_000.0  # Dünya yarıçapı (metre)
    lat_pred_rad = lat_pred * (math.pi / 180.0)
    lon_pred_rad = lon_pred * (math.pi / 180.0)
    lat_true_rad = lat_true * (math.pi / 180.0)
    lon_true_rad = lon_true * (math.pi / 180.0)

    dlat = lat_true_rad - lat_pred_rad
    dlon = lon_true_rad - lon_pred_rad

    a = torch.sin(dlat / 2)**2 + torch.cos(lat_pred_rad) * torch.cos(lat_true_rad) * torch.sin(dlon / 2)**2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1 - a))

    distances = R * c
    return distances.mean()

#######################
# 3) CUSTOM LOSS
#######################
mse_criterion = nn.MSELoss()

def combined_loss(outputs, targets, alpha=0.95):
    # 1) MSE (normalleştirilmiş lat/lon)
    mse = mse_criterion(outputs, targets)

    # 2) Haversine
    lat_pred_norm = outputs[:, 0]
    lon_pred_norm = outputs[:, 1]
    lat_true_norm = targets[:, 0]
    lon_true_norm = targets[:, 1]

    lat_pred_deg, lon_pred_deg = inverse_zscore_lat_lon(lat_pred_norm, lon_pred_norm)
    lat_true_deg, lon_true_deg = inverse_zscore_lat_lon(lat_true_norm, lon_true_norm)

    haversine_m = haversine_distance_tensor(lat_pred_deg, lon_pred_deg, lat_true_deg, lon_true_deg)
    total_loss = alpha * mse + (1 - alpha) * (haversine_m / 1000.0)
    return total_loss

#########################
# 4) DATASET TANIMI
#########################
class CampusDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, use_zscore=True):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.use_zscore = use_zscore

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Buraya debug print konulabilir ama çok sık olur => yavaşlatır
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.img_dir, row["filename"])
        lat = float(row["latitude"])
        lon = float(row["longitude"])
        
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        if self.use_zscore:
            lat, lon = zscore_lat_lon(lat, lon)
        
        target = torch.tensor([lat, lon], dtype=torch.float32)
        return image, target

#########################
# 5) MODEL TANIMI
#########################
class TransformerRegressor(nn.Module):
    """
    Tek bir model sınıfı ile DeiT, Swin, BEiT, ViT vs. AutoModel ile yükleyebilirsiniz.
    """
    def __init__(self, model_name="microsoft/swin-base-patch4-window7-224", output_dim=2):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.backbone = AutoModel.from_pretrained(model_name, config=self.config)

        hidden_size = self.config.hidden_size
        self.head = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        outputs = self.backbone(x)
        if hasattr(outputs, 'last_hidden_state'):
            cls_emb = outputs.last_hidden_state[:, 0, :]
        else:
            cls_emb = outputs.pooler_output
        preds = self.head(cls_emb)
        return preds

#########################
# 6) OPTUNA OBJECTIVE
#########################
def objective(trial, train_dataset, device):
    print("\n[Optuna] => Yeni deneme (trial) başlıyor...")
    alpha = trial.suggest_float("alpha", 0.90, 0.99)
    lr = trial.suggest_loguniform("lr", 1e-5, 5e-4)

    model_name = "microsoft/swin-base-patch4-window7-224"

    n_total = len(train_dataset)
    n_val = int(0.2 * n_total)
    n_train = n_total - n_val

    print(f"[Optuna] => Dataset toplam: {n_total}, Train: {n_train}, Val: {n_val}")
    train_ds, val_ds = random_split(train_dataset, [n_train, n_val])

    # Burada num_workers=4 => Dağılımlar
    # pin_memory=True => bazen hız, bazen yavaşlama. Duruma göre değiştirip test edebilirsiniz.
    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
    val_loader   = DataLoader(val_ds,   batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

    model = TransformerRegressor(model_name=model_name, output_dim=2).to(device)
    if torch.cuda.device_count() > 1:
        print(f"[Optuna] => {torch.cuda.device_count()} GPU var, DataParallel aktif.")
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

    # CosineAnnealingLR (3 epoch => T_max=3)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=3, eta_min=1e-6)

    scaler = GradScaler()

    EPOCHS = 3
    for epoch in range(EPOCHS):
        print(f"[Optuna] => Epoch {epoch+1}/{EPOCHS} başlıyor...")
        model.train()
        t0 = time.time()
        running_loss = 0.0

        # Eklemek istediğimiz: train_loader üzerinde tqdm ile batch'leri gösterelim
        with tqdm(total=len(train_loader), desc=f"OptunaTrain E{epoch+1}", unit="batch") as pbar:
            for batch_idx, (images, targets) in enumerate(train_loader):
                images, targets = images.to(device), targets.to(device)

                optimizer.zero_grad()
                with autocast():
                    outputs = model(images)
                    loss = combined_loss(outputs, targets, alpha=alpha)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                running_loss += loss.item()
                pbar.set_postfix({"loss": f"{running_loss/(batch_idx+1):.4f}"})
                pbar.update(1)

        scheduler.step()
        print(f"[Optuna] => Epoch {epoch+1} bitti, süresi: {time.time()-t0:.2f} sn")

    # Validation
    print("[Optuna] => Validation aşaması başlıyor...")
    model.eval()
    val_loss = 0.0
    with torch.no_grad(), autocast():
        with tqdm(total=len(val_loader), desc="Val", unit="batch") as pbar2:
            for batch_idx, (images, targets) in enumerate(val_loader):
                images, targets = images.to(device), targets.to(device)
                preds = model(images)
                loss = combined_loss(preds, targets, alpha=alpha)
                val_loss += loss.item()
                pbar2.update(1)

    val_loss /= len(val_loader)
    print(f"[Optuna] => Validation loss: {val_loss:.4f}")
    return val_loss

#########################
# 7) ANA EĞİTİM FONKSİYONU
#########################
print("=== Başlangıç ===")
CSV_FILE  = "/kaggle/input/hulyais/train.csv"
IMG_DIR   = "/kaggle/input/hulyais/train"

print("CSV okunuyor...")
df = pd.read_csv(CSV_FILE, sep=';')
print(f"CSV satır sayısı: {len(df)}")

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

print("Dataset oluşturuluyor...")
dataset = CampusDataset(
    dataframe=df,
    img_dir=IMG_DIR,
    transform=train_transform,
    use_zscore=True
)
print(f"Dataset length: {len(dataset)}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}, GPU Sayısı: {torch.cuda.device_count()}")

#########################
# 7.1) OPTUNA TUNING
#########################
def optuna_objective(trial):
    return objective(trial, dataset, device)

print("=== Optuna Study oluşturuluyor ===")
study = optuna.create_study(direction="minimize")
print("=== Optuna Study optimize başlıyor ===")
study.optimize(optuna_objective, n_trials=2)  # n_trials=2 test amaçlı, artırabilirsiniz.

print("Optuna tamamlandı! Best params:")
print(study.best_params)
print("Best value (val_loss):", study.best_value)

alpha_best = study.best_params["alpha"]
lr_best = study.best_params["lr"]

#########################
# 7.2) NİHAİ EĞİTİM
#########################
print("\n=== Nihai eğitim başlıyor ===")
train_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)

final_model_name = "microsoft/swin-base-patch4-window7-224"
model = TransformerRegressor(model_name=final_model_name, output_dim=2).to(device)
if torch.cuda.device_count() > 1:
    print(f"Nihai eğitim: {torch.cuda.device_count()} GPU var, DataParallel aktif.")
    model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))

optimizer = optim.AdamW(model.parameters(), lr=lr_best, weight_decay=1e-4)

# CosineAnnealingLR (5 epoch => T_max=5)
FINAL_EPOCHS = 25
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=FINAL_EPOCHS, eta_min=1e-6)

scaler = GradScaler()

for epoch in range(FINAL_EPOCHS):
    print(f"[Nihai] => Epoch {epoch+1}/{FINAL_EPOCHS} başlıyor...")
    model.train()
    running_loss = 0.0
    t_start = time.time()

    with tqdm(total=len(train_loader), desc=f"NihaiTrain E{epoch+1}", unit="batch") as pbar:
        for batch_idx, (images, targets) in enumerate(train_loader):
            images, targets = images.to(device), targets.to(device)

            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = combined_loss(outputs, targets, alpha=alpha_best)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            pbar.set_postfix({"loss": f"{running_loss/(batch_idx+1):.4f}"})
            pbar.update(1)

    scheduler.step()
    epoch_loss = running_loss / len(train_loader)
    print(f"[Nihai] => Epoch {epoch+1} bitti. Loss: {epoch_loss:.4f}, Süre: {time.time()-t_start:.2f} sn, LR: {scheduler.get_last_lr()}")

print("Nihai eğitim tamamlandı.")
torch.save(model.state_dict(), "/kaggle/working/final_model.pth")
print("Model kaydedildi: /kaggle/working/final_model.pth")



"""
final 8 için 0.21 verdi

"""

[I 2024-12-22 03:18:12,831] A new study created in memory with name: no-name-53176996-53bf-4cfe-ae76-fbbaf63dd725


=== Başlangıç ===
CSV okunuyor...
CSV satır sayısı: 4375
Dataset oluşturuluyor...
Dataset length: 4375
Device: cuda, GPU Sayısı: 2
=== Optuna Study oluşturuluyor ===
=== Optuna Study optimize başlıyor ===

[Optuna] => Yeni deneme (trial) başlıyor...
[Optuna] => Dataset toplam: 4375, Train: 3500, Val: 875


<ipython-input-2-b26102738f1a>:153: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 5e-4)


config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/352M [00:00<?, ?B/s]

...

[Nihai] => Epoch 23 bitti. Loss: 0.0616, Süre: 405.46 sn, LR: [1.5863248700807082e-06]
[Nihai] => Epoch 24/25 başlıyor...


NihaiTrain E24: 100%|██████████| 274/274 [06:47<00:00,  1.49s/batch, loss=0.0597]


[Nihai] => Epoch 24 bitti. Loss: 0.0597, Süre: 407.54 sn, LR: [1.1471614234094626e-06]
[Nihai] => Epoch 25/25 başlıyor...


NihaiTrain E25: 100%|██████████| 274/274 [06:42<00:00,  1.47s/batch, loss=0.0593]


[Nihai] => Epoch 25 bitti. Loss: 0.0593, Süre: 402.93 sn, LR: [1e-06]
Nihai eğitim tamamlandı.
Model kaydedildi: /kaggle/working/final_model.pth


In [3]:
import torch

# Kaydedilmiş checkpoint yükle
saved_state_dict = torch.load("final_model.pth", map_location="cpu")

# Yeni sözlük oluşturup 'module.' prefixini ayıkla
new_state_dict = {}
for k, v in saved_state_dict.items():
    if k.startswith("module."):
        new_key = k[len("module."):]  # 'module.' uzunluğu kadar kes
    else:
        new_key = k
    new_state_dict[new_key] = v

# Şimdi new_state_dict parametre adları 'module.' olmadan
torch.save(new_state_dict, "final_model_no_module.pth")
print("Yeni checkpoint kaydedildi: final_model_no_module.pth")


<ipython-input-3-07db80e46eb1>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load("final_model.pth", map_location="cpu")


Yeni checkpoint kaydedildi: final_model_no_module.pth


In [4]:
##########################################
# TEST / INFERENCE KODU (ÖZET)
##########################################
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoConfig
from torchvision import transforms
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm

# 1) Z-Score değerleri, inverse fonksiyon vs.
LAT_MEAN = 41.105763
LAT_STD  = 0.002358
LON_MEAN = 29.025191
LON_STD  = 0.004205
def inverse_zscore_lat_lon(lat_norm, lon_norm):
    lat_real = lat_norm * LAT_STD + LAT_MEAN
    lon_real = lon_norm * LON_STD + LON_MEAN
    return lat_real, lon_real

# 2) TestDataset
class TestDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.img_dir, row["filename"])
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image


TEST_CSV = "/kaggle/input/hulyais/test.csv"
TEST_DIR = "/kaggle/input/hulyais/test"

test_df = pd.read_csv(TEST_CSV, sep=';')

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
test_dataset = TestDataset(test_df, TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model oluştur
model = TransformerRegressor("microsoft/swin-base-patch4-window7-224", output_dim=2)
model.load_state_dict(torch.load("/kaggle/working/final_model_no_module.pth", map_location=device))

# 2 GPU istersek (opsiyonel)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))

model.to(device)
model.eval()

predictions_lat = []
predictions_lon = []

with torch.no_grad():
    for images in tqdm(test_loader, desc="Inference", unit="batch"):
        images = images.to(device)
        outputs = model(images)  # (batch_size, 2)
        lat_norm = outputs[:, 0]
        lon_norm = outputs[:, 1]
        lat_vals, lon_vals = inverse_zscore_lat_lon(lat_norm, lon_norm)
        lat_vals = lat_vals.cpu().numpy()
        lon_vals = lon_vals.cpu().numpy()

        predictions_lat.extend(lat_vals)
        predictions_lon.extend(lon_vals)

test_df["latitude"] = predictions_lat
test_df["longitude"] = predictions_lon

out_path = "/kaggle/working/test_predictions.csv"
test_df.to_csv(out_path, index=False)
print(f"Tahminler kaydedildi: {out_path}")



<ipython-input-4-7380b393d2d9>:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/working/final_model_no_module.pth", map_location=d

Tahminler kaydedildi: /kaggle/working/test_predictions.csv
